# UDACITY Demo - Step Functions

In [ ]:
import boto3
from sagemaker import get_execution_role

client = boto3.client('stepfunctions')

# Definitions are unweildly and also not valid python objects. Consider also using the Step interface
# for the services you wish to invoke, or just constructing a Step Function through the UI 
# and pasting the definition that is automatically generated. 

definition = """{
  "Comment": "A Hello World example of the Amazon States Language.",
  "StartAt": "Lambda Invoke (First)", 
  "States": {
    "Lambda Invoke (First)": {
      "Type": "Task",
      "Resource": "arn:aws:states:::lambda:invoke",
      "OutputPath": "$.Payload",
      "Parameters": {
        "Payload.$": "$",
        "FunctionName": "arn:aws:lambda:us-west-2:565094796913:function:example123:$LATEST"
      },
      "Retry": [
        {
          "ErrorEquals": [
            "Lambda.ServiceException",
            "Lambda.AWSLambdaException",
            "Lambda.SdkClientException"
          ],
          "IntervalSeconds": 2,
          "MaxAttempts": 6,
          "BackoffRate": 2
        }
      ],
      "Next": "Lambda Invoke (Second)"
    },
    "Lambda Invoke (Second)": {
      "Type": "Task",
      "Resource": "arn:aws:states:::lambda:invoke",
      "OutputPath": "$.Payload",
      "Parameters": {
        "Payload.$": "$",
        "FunctionName": "arn:aws:lambda:us-west-2:565094796913:function:example123:$LATEST"
      },
      "Retry": [
        {
          "ErrorEquals": [
            "Lambda.ServiceException",
            "Lambda.AWSLambdaException",
            "Lambda.SdkClientException"
          ],
          "IntervalSeconds": 2,
          "MaxAttempts": 6,
          "BackoffRate": 2
        }
      ],
      "End": true
    }
  }
}
"""

In [ ]:
# Your SageMaker execution role needs to have both StepFunctions permission and IAM permission. This
# is seperate from the role passed in through 'roleArn' (You need IAM permissions specifically because
# you are handling other roles.) The roleArn that's passed into the state machine needs to have permissions
# to the services it's invoking. In this case, these are Lambda permissions. 

response = client.create_state_machine(
    name='boto3StateMachine3', # Names need to be unique. 
    definition=definition,
    roleArn='arn:aws:iam::565094796913:role/service-role/StepFunctions-firstStateMachine-role-0826984a',
    type='STANDARD',
    loggingConfiguration={
        'level': 'OFF'
    }
)
print(response)

In [ ]:
response = client.start_execution(
    stateMachineArn='arn:aws:states:us-west-2:565094796913:stateMachine:boto3StateMachine3', # You can find this through the Console or through the 'response' object. 
    name='example1', # Execution names need to be unique within state machines. 
    input='{}' # Input needs to be at least empty brackets. 
)